<h4>Transfer Learning</h4>

<h5>Imports</h5>

In [8]:
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, unquote_plus
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Input, Dropout
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Layer
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
import jsonlines
import os
import json
import pickle
from sklearn.metrics import f1_score, accuracy_score, precision_score
from typing import Callable
from sklearn import svm
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from tensorflow.python.profiler import profiler_v2 as profiler

2023-05-15 09:35:23.408775: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-15 09:35:23.523284: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 09:35:24.714317: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [22]:
def parse_http_requests(file_path):
    requests = []
    current_request = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            line = line.strip()
            if line == '':
                if current_request:
                    print(current_request)
                    if (current_request['method'] == 'POST' or current_request['method'] == 'PUT') and 'body' not in current_request.keys():
                        print("WWWW")
                        continue    
                    requests.append(current_request)
                    current_request = {}
            elif line.startswith('GET') or line.startswith('POST') or line.startswith('PUT'):
                method, uri, _ = line.split(' ')
                current_request['method'] = method
                current_request['uri'] = uri
            else:
                if ': ' not in line:
                    current_request['body'] = line
                else:
                    header_name, header_value = line.split(': ', 1)
                    if 'headers' not in current_request:
                        current_request['headers'] = {}
                    current_request['headers'][header_name] = header_value
    if current_request:
        requests.append(current_request)
    return requests


file_paths = ['./dataset/cisc/normalTrafficTraining.txt', './dataset/cisc/anomalousTrafficTest.txt', './dataset/cisc/normalTrafficTest.txt']
cisc_train = parse_http_requests(file_paths[0])
cisc_test_malicious = parse_http_requests(file_paths[1])
cisc_test_normal = parse_http_requests(file_paths[2])

def parse_requests(requests):
    data=[]
    for request in requests:
        if request['method'] == 'POST':
            print(request)
            data.append('POST'+' '+request['uri']+' '+request['body']+' '+json.dumps(request['headers']))
        else:
            data.append('GET'+' '+request['uri']+' '+json.dumps(request['headers']))
    return data

train_examples = parse_requests(cisc_train)
normal = parse_requests(cisc_test_normal)
malicious = parse_requests(cisc_test_malicious)
test_examples = normal+malicious
train_labels = [0] * len(train_examples)
test_labels = [0]* len(normal)
test_labels.extend([1] * len(malicious))

dataset_train = tf.data.Dataset.from_tensor_slices((list(map(lambda x: unquote_plus(x),train_examples)), train_labels))
dataset_test = tf.data.Dataset.from_tensor_slices((list(map(lambda x: unquote_plus(x),test_examples)), test_labels))

#shuflee datasets with given radnom seed
dataset_train = dataset_train.shuffle(400000, seed=42, reshuffle_each_iteration=False)
dataset_test = dataset_test.shuffle(400000, seed=42, reshuffle_each_iteration=False)

def preprocess_text(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[a-zA-Z]+", "a")
    text = tf.strings.regex_replace(text, "[0-9]+", "n")
    text = tf.strings.regex_replace(text, "(a|n){2,}", "x")
    text = tf.strings.regex_replace(text, '[^\x00-\x7F]+', '')
    text = tf.strings.regex_replace(text, '(.)',  r'\1 ')
    return text, label

def preprocess_text_substitution(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[a-zA-Z]+", "a")
    text = tf.strings.regex_replace(text, "[0-9]+", "n")
    text = tf.strings.regex_replace(text, "(a|n){2,}", "x")
    text = tf.strings.regex_replace(text, '[^\x00-\x7F]+', '')
    punctuation = "=?/(){}[]<>"
    for p in punctuation:
        text = tf.strings.regex_replace(text, "\\" + p, " "+p+" ")
    return text, label

 
# Map the preprocess function to the dataset
train = dataset_train.map(preprocess_text)
test = dataset_test.map(preprocess_text)

train_sub = dataset_train.map(preprocess_text_substitution)
test_sub = dataset_test.map(preprocess_text_substitution)

for text, label in train_sub.take(1):
    print(text.numpy(), label.numpy())

for text, label in train.take(1):
    print(text.numpy(), label.numpy())


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'method': 'POST', 'uri': 'http://localhost:8080/tienda1/publico/registro.jsp', 'headers': {'User-Agent': 'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko)', 'Pragma': 'no-cache', 'Cache-control': 'no-cache', 'Accept': 'text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5', 'Accept-Encoding': 'x-gzip, x-deflate, gzip, deflate', 'Accept-Charset': 'utf-8, utf-8;q=0.5, *;q=0.5', 'Accept-Language': 'en', 'Host': 'localhost:8080', 'Cookie': 'JSESSIONID=F170D2D69063981B08C7348A06C58DBE', 'Content-Type': 'application/x-www-form-urlencoded', 'Connection': 'close', 'Content-Length': '315'}, 'body': 'modo=registro&login=teena&password=EnraNCiAR&nombre=Julio&apellidos=Guardiol+Sim%F3&email=heuser%40prensahp.mx&dni=31293475C&direccion=Calle+Rejas%2C+23%2C+4%3FB&ciudad=Montiel&cp=08183&provincia=paros%2522%2520style%3D%2522background%3Aurl%28javascript%3Aalert%28%27Paros%27%29%29&ntc=5996994438589439&B1=Registrar'}
{'method': 'PO

2023-05-15 10:35:32.361435: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-15 10:35:32.560293: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-15 10:35:32.560354: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-15 10:35:32.571894: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-15 10:35:32.571979: I tensorflow/compile

b'a a: /  / a:n / x / a / n.a  { "a-a": "a / n.n  ( a; a / n.n; a )  a / n.n.n  ( a a ) ", "a": "a-a", "a-a": "a-a", "a": "a / a,a / a,a / a a,a / a;a = n.n,a / a;a = n.n,a / a,* / *;a = n.n", "a-a": "a-a, a-a, a, a", "a-a": "a-n, a-n;a = n.n, *;a = n.n", "a-a": "a", "a": "a:n", "a": "a = x", "a": "a" } ' 0
b'a   a : / / a : n / x / a / n . a   { " a - a " :   " a / n . n   ( a ;   a / n . n ;   a )   a / n . n . n   ( a   a ) " ,   " a " :   " a - a " ,   " a - a " :   " a - a " ,   " a " :   " a / a , a / a , a / a   a , a / a ; a = n . n , a / a ; a = n . n , a / a , * / * ; a = n . n " ,   " a - a " :   " a - a ,   a - a ,   a ,   a " ,   " a - a " :   " a - n ,   a - n ; a = n . n ,   * ; a = n . n " ,   " a - a " :   " a " ,   " a " :   " a : n " ,   " a " :   " a = x " ,   " a " :   " a " } ' 0


2023-05-15 10:35:38.487222: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [36000]
	 [[{{node Placeholder/_1}}]]
2023-05-15 10:35:38.587242: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [36000]
	 [[{{node Placeholder/_1}}]]


<h4>Save and load vectorization from disk</h4>

In [6]:

from_disk = pickle.load(open("vectorization.pkl", "rb"))
vectorization = TextVectorization.from_config(from_disk['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorization.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorization.set_weights(from_disk['weights'])

from_disk_sub = pickle.load(open("vectorization_sub.pkl", "rb"))
vectorization_sub = TextVectorization.from_config(from_disk_sub['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorization_sub.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorization_sub.set_weights(from_disk_sub['weights'])

2023-05-15 04:27:18.015052: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-05-15 04:27:18.126171: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


<h4>Load and retrain all models</h4>

In [6]:
#load lstm model
lstm_model = tf.keras.models.load_model('lstm/my_model')

#load transformer model
transformer_model = tf.keras.models.load_model('transformer/my_model')

#load cnn model
cnn_model = tf.keras.models.load_model('cnn/my_model') 

#load autoencoder model
autoencoder_model = tf.keras.models.load_model('autoencoder/my_model')

#create new svm model   
svm_model = svm.OneClassSVM(gamma='auto')


Epoch 1/4


2023-05-15 03:28:07.798189: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [211600]
	 [[{{node Placeholder/_0}}]]
2023-05-15 03:28:10.396317: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-15 03:28:10.406718: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0xe095940 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-15 03:28:10.406750: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2023-05-15 03:28:10.412948: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash r

6613/6613 [==============================] - 95s 14ms/step - loss: 0.2428
Epoch 2/4
6613/6613 [==============================] - 94s 14ms/step - loss: 0.1854
Epoch 3/4
6613/6613 [==============================] - 94s 14ms/step - loss: 0.1619
Epoch 4/4
6613/6613 [==============================] - 94s 14ms/step - loss: 0.1299


In [16]:
trans.fit(train_auto, epochs=6)
trans.save('trans/my_model')
trans = tf.keras.models.load_model('trans/my_model')


Epoch 1/6


2023-05-15 03:51:36.258923: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall' with dtype float and shape [?,256,32]
	 [[{{node gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall}}]]
2023-05-15 03:51:36.259037: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_1' with dtype float and shape [?,256,32]
	 [[{{node gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_1}}]]
2023-05-15 03:51:36.259072: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Ex

6613/6613 [==============================] - 90s 13ms/step - loss: 0.0699
Epoch 2/6
6613/6613 [==============================] - 92s 14ms/step - loss: 0.0654
Epoch 3/6
6613/6613 [==============================] - 96s 14ms/step - loss: 0.0610
Epoch 4/6
6613/6613 [==============================] - 97s 15ms/step - loss: 0.0579
Epoch 5/6
6613/6613 [==============================] - 94s 14ms/step - loss: 0.0559
Epoch 6/6
6613/6613 [==============================] - 94s 14ms/step - loss: 0.0534


2023-05-15 04:00:59.683888: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'dense_input' with dtype float and shape [?,?,32]
	 [[{{node dense_input}}]]
2023-05-15 04:00:59.701713: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'dense_input' with dtype float and shape [?,?,32]
	 [[{{node dense_input}}]]
2023-05-15 04:00:59.709340: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,32]
	 [[{{node inpu

INFO:tensorflow:Assets written to: trans/my_model/assets


INFO:tensorflow:Assets written to: trans/my_model/assets


In [7]:
trans = tf.keras.models.load_model('trans/my_model')
def autoencoder_processing(text, label):
    return  vectorization(text), label
test_auto = test_sub.map(autoencoder_processing)
test_auto = test_auto.batch(32)
reconsturctions =  trans.predict(test_auto)


2023-05-15 04:27:18.859121: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [117899]
	 [[{{node Placeholder/_1}}]]
2023-05-15 04:27:18.970361: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'x' with dtype float and shape [?,?]
	 [[{{node x}}]]


  21/3685 [..............................] - ETA: 28s  

2023-05-15 04:27:20.029074: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3685/3685 [==============================] - 25s 6ms/step


In [8]:
reconsturctions

array([[0.9905336 ],
       [0.01712406],
       [0.9998777 ],
       ...,
       [0.99995923],
       [0.9997352 ],
       [0.00318526]], dtype=float32)

In [9]:
pred = list(map(lambda x: 0 if x[0]<0.5 else 1, list(reconsturctions)))

In [ ]:
actual =test_auto.map(lambda x,y:y)
actual = np.concatenate(list(actual.as_numpy_iterator()))


2023-05-15 04:27:45.155001: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype string
	 [[{{node Placeholder/_11}}]]


In [ ]:
f1_score(actual,pred)

In [24]:
pred

[1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
